In [1]:
import onnx
import torch
onnx_model = onnx.load("retinanet_0306_11_simplify.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

img = Image.open("demo/demo.jpg")

resize = transforms.Resize([224, 224])
img = resize(img)

to_tensor = transforms.ToTensor()
img = to_tensor(img)
img.unsqueeze_(0)

In [4]:
import onnxruntime
ort_session = onnxruntime.InferenceSession("retinanet_0306_11_simplify.onnx")

In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [6]:
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_cls = ort_outs[0]
img_out_cor = ort_outs[5]

In [7]:
img_out_cls = torch.from_numpy(img_out_cls)
img_out_cor = torch.from_numpy(img_out_cor)
img_out_cls = img_out_cls[0]
img_out_cor = img_out_cor[0]

In [8]:
img_out_cls

tensor([[[ -4.8286,  -5.9745,  -6.2251,  ...,  -6.5255,  -6.2767,  -4.9740],
         [ -5.7057,  -7.0122,  -7.1242,  ...,  -7.4001,  -7.3419,  -5.9367],
         [ -5.4860,  -6.4135,  -6.3247,  ...,  -6.3943,  -6.6490,  -5.7228],
         ...,
         [ -7.2285,  -8.7930,  -8.6530,  ...,  -6.8451,  -7.3982,  -6.5985],
         [ -6.6097,  -8.0864,  -8.1054,  ...,  -6.9062,  -7.3005,  -6.3032],
         [ -5.2292,  -6.3495,  -6.4406,  ...,  -6.1062,  -6.2506,  -5.2537]],

        [[ -5.0633,  -6.4914,  -6.8168,  ...,  -7.4418,  -7.2737,  -5.7266],
         [ -5.9852,  -7.6524,  -7.8858,  ...,  -8.7042,  -8.6110,  -6.7075],
         [ -5.9918,  -7.4485,  -7.4833,  ...,  -8.3865,  -8.5175,  -6.7510],
         ...,
         [ -8.0737, -10.5846, -10.9362,  ...,  -9.3460,  -9.7349,  -7.8240],
         [ -7.6191, -10.0256, -10.5513,  ...,  -9.3258,  -9.4385,  -7.4494],
         [ -6.1253,  -7.8786,  -8.3287,  ...,  -7.8753,  -7.7175,  -6.0023]],

        [[ -4.5669,  -5.6121,  -5.8245,  ...

In [9]:
img_out_cor

tensor([[[-1.5369e-02, -9.2218e-03, -3.4875e-02,  ...,  7.1130e-02,
           6.4021e-02, -6.8020e-04],
         [-1.7606e-02,  3.3459e-03, -6.6329e-02,  ...,  1.7323e-01,
           1.0249e-01, -1.2830e-02],
         [ 2.8835e-03,  2.2703e-03, -8.6033e-02,  ...,  1.8347e-01,
           1.1363e-01,  8.2252e-03],
         ...,
         [-8.7731e-02, -5.2767e-02, -5.8413e-02,  ..., -6.1124e-02,
          -1.4057e-01, -2.0284e-02],
         [-4.7661e-02, -1.7821e-02, -5.0728e-02,  ..., -4.2439e-02,
          -1.2349e-01, -4.1876e-02],
         [ 2.8583e-03, -1.0518e-02, -4.7419e-02,  ..., -5.0873e-02,
          -8.0268e-02, -6.7537e-02]],

        [[ 3.9942e-02,  7.4324e-02,  8.7586e-02,  ...,  1.0182e-01,
           1.4292e-01,  1.0591e-01],
         [-6.3956e-03, -3.1180e-05, -1.4447e-02,  ..., -4.7931e-02,
          -2.9105e-02, -1.5479e-02],
         [ 2.3332e-02,  3.0228e-02,  1.8079e-02,  ..., -8.5370e-02,
          -1.1372e-01, -8.6410e-02],
         ...,
         [ 3.0552e-02,  6

In [13]:
for i in range(28):
    for j in range(28):
        for m in range(720):
            if img_out_cls[ m , i, j] > 0.05:
                print(i, '/', j)

8 / 11
9 / 11
9 / 12


In [14]:
cls = img_out_cls[:, 9, 11]

In [17]:
for i in range(720):
    if cls[i] > 0:
        print(i)

82


In [18]:
cor = img_out_cor[:, 9, 11]

In [19]:
cor

tensor([-0.0222, -0.1321,  0.1845,  0.0960,  0.0471, -0.1094,  0.1168, -0.1234,
         0.0755, -0.0856, -0.0375, -0.3335, -0.1205, -0.0813,  0.3277, -0.2528,
        -0.0288, -0.0737,  0.3104, -0.4112,  0.0581, -0.0611,  0.2258, -0.5191,
        -0.0907, -0.0644,  0.2392, -0.4909, -0.1212, -0.0491,  0.2224, -0.4624,
        -0.0707, -0.0252,  0.1956, -0.4937])

torch.Size([3, 100, 4])
